In [ ]:
#---------------------------
# imports
#---------------------------
import os
from glob import glob
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm
tqdm.pandas()
#---------------------------
# helpers
#---------------------------
punctuations           =    ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', 
                            '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`',
                            '{', '|', '}', '~', '।', '॥','“','”'] 
def reset(df):
    df.dropna(inplace=True)
    df.reset_index(drop=True,inplace=True) 
    return df

def get_words(text):
    if text.strip():
        for punct in punctuations+["\n"]:
            text=text.replace(punct," ")
        words=[word for word in text.split(" ") if word.strip()]
        return Counter(words)
    else:
        return None

def cvtCounter(counter):
    df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
    df = df.rename(columns={'index':'word', 0:'count'})
    return df

def split_dataframe(df, chunk_size = 1000): 
    chunks =[]
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def process_csv(csv,didx,words_dir):
    df=pd.read_csv(csv)
    df["words"]=df.text.progress_apply(lambda x:get_words(x))
    df=reset(df)
    chunks=split_dataframe(df)
    for cidx,df in enumerate(chunks):
        data=Counter()
        for idx in tqdm(range(len(df))):
            data+=df.iloc[idx,-1]
        data=cvtCounter(data)
        _csv=os.path.join(words_dir,f"{didx}_{cidx}.csv")
        data.to_csv(_csv,index=False) 


In [ ]:
for lang in ["bn","hi","ml","gu","ta","pa","or"]:
    data_dir=f"/backup/Oscar/data/{lang}"
    words_dir=f"/backup/Oscar/words/{lang}/"
    if not os.path.exists(words_dir):
        os.mkdir(words_dir)
        
    csvs=[csv for csv in glob(os.path.join(data_dir,"*.csv"))]
    for didx,csv in enumerate(csvs):
        print(lang,didx)
        process_csv(csv,didx,words_dir)



In [ ]:
#---------------------------
# imports
#---------------------------
import re
import os
from glob import glob
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
#---------------------------
# csv
#---------------------------
def reset(df):
    df.dropna(inplace=True)
    df.reset_index(drop=True,inplace=True) 
    return df

def de_emojify(text):
    regrex_pattern = re.compile(pattern = "["u"\U0001F600-\U0001F64F"  # emoticons
                                             u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                             u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                             u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                            "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean(text):
    text=str(text)
    text = re.sub("[a-zA-Z0-9]+", "",text)
    if len(text)==0:
        return None
    else:
        text=de_emojify(text)
        if len(text)>0:
            return text
        else:
            return None

lang="bn"
data_dir=f"/backup/Oscar/words/{lang}/"
csvs=[csv for csv in glob(os.path.join(data_dir,"*.csv"))]
dfs=[pd.read_csv(csv) for csv in tqdm(csvs)]
df=pd.concat(dfs,ignore_index=True)
df=reset(df)
df["word"]=df["word"].progress_apply(lambda x:clean(x))
df=reset(df)
# group
df_new = df.groupby(df['word']).aggregate({"count":"sum"})
df_new = df_new.sort_values(by='count', ascending=False)
# new
df=pd.DataFrame({})
df["word"]=df_new.index.tolist()
df["count"]=df_new["count"].tolist()
df["count"]=df["count"].progress_apply(lambda x:int(x))
df.to_csv(os.path.join("/backup/Oscar/words/",f"{lang}.csv"))
df